## P1 - PREPROCESAMIENTO

In [131]:
import nltk
from nltk.stem import SnowballStemmer

# Cargar el archivo de stoplist [CON ESTE TRABAJAREMOS countwords]
with open('stoplist.txt', 'r') as file:
    stoplist = file.read().splitlines()
stoplist += ['.', ',', ';', ':', '!', '?', '¿', '¡', '(', ')', '[', ']', '{', '}', '"', "'", '``', "''","111âº"]

lexema = SnowballStemmer('spanish')

# Preprocesamiento
def preprocesamiento(doc):
    file = open(doc, "r")
    # 1 - tokenizar
    words = nltk.word_tokenize(file.read())
    # 2 - normalizar
    words = [word.lower() for word in words]
    # 3 - reducción (stemming)
    words = [lexema.stem(word) for word in words]
    # 4 - eliminar stopwords
    words = [word for word in words if word not in stoplist]
    return words

# Muestra de preprocesamiento
preprocesamiento('libro1.txt')[0:10]

['obra',
 'comienz',
 'notici',
 'celebraciã³n',
 'cumpleaã±',
 'bilb',
 'bolsã³n',
 'comarc',
 'embarg',
 'par']

## P2 - CONSTRUCCIÓN DE ÍNDICE INVERTIDO DE LOS 500 TÉRMINOS MÁS COMUNES

In [136]:
# Indice invertido
from collections import defaultdict
from collections import Counter

# saquemos las 500 palabras más frecuentes
lista_libros= []
lista_libros.append(preprocesamiento('libro1.txt'))
lista_libros.append(preprocesamiento('libro2.txt'))
lista_libros.append(preprocesamiento('libro3.txt'))
lista_libros.append(preprocesamiento('libro4.txt'))
lista_libros.append(preprocesamiento('libro5.txt'))
lista_libros.append(preprocesamiento('libro6.txt'))

word_freq = Counter()
# Contar la frecuencia de cada palabra en cada libro preprocesado
for text in lista_libros:
    word_freq.update(text)
    
most_commom_words = word_freq.most_common(500)
most_commom_words = [word for word, freq_word in most_commom_words]
list_docs = ['libro1.txt','libro2.txt','libro3.txt','libro4.txt','libro5.txt','libro6.txt']

# a) construir el índice invertido de las 500 palabras más frecuentes [lexemas]
def indice_invertido_common_words(docs, most_common_words):
    index = defaultdict(lambda: defaultdict(int))
    for i, doc in enumerate(docs):
        words = preprocesamiento(doc)
        for word in words:
            if word in most_common_words:
                index[word][i+1] += 1

    # Convertir el índice a listas ordenadas de tuplas
    for word, doc_freqs in index.items():
        index[word] = sorted(doc_freqs.items())
        
    index = sorted(index.items())
    
    # Convertir el índice a una cadena de texto
    index_str = ""
    for word, doc_freqs in index:
        index_str += f"{word},{len(doc_freqs)} -> "
        index_str += " -> ".join(f"{doc_id}" for doc_id, _ in doc_freqs)
        index_str += "\n"

    # Guardar el índice en un archivo de texto
    with open("indice_invertido_500_FLECHA.txt", "w") as file:
        file.write(index_str)
    
    return index

#Muestra de índice invertido de las 500 palabras más frecuentes
print(indice_invertido_common_words(list_docs, most_commom_words)[0:3])
        
# b) guardar el índice en un archivo de texto[lexemas]
def guardar_indice_invertido(docs, most_common_words):
    index = indice_invertido_common_words(docs, most_common_words)
    # Convertir el índice a una cadena de texto
    index_str = ""
    for word, doc_freqs in index:
        index_str += f"{word}: "
        index_str += ", ".join(f"{doc_id}" for doc_id, _ in doc_freqs)
        index_str += "\n"
    with open('indice_invertido_500_COMAS.txt', 'w') as file:
        file.write(index_str)
        
guardar_indice_invertido(list_docs,most_commom_words)

[('abism', [(2, 1), (3, 2)]), ('acab', [(1, 3), (4, 2), (5, 1), (6, 2)]), ('acabã³', [(1, 1)])]


## P3 - APLICAR CONSULTAS BOOLEANAS

In [135]:
# consultas booleanas

list_libros = [1, 2, 3, 4, 5, 6] # libros del 1 al 6

lexema = SnowballStemmer('spanish')

def L(word): # busca en que libros se encuentra el lexema
    result = []
    for i, text in enumerate(lista_libros, start=1):
        # si el lexema está en el texto (ej : lexema = comun => comunidad SI APARECE)
        if lexema.stem(word) in text:
            result.append(i)
    return result

def AND(A, B): # retorna los libros en los que se encuentran ambas palabras
    i, j = 0, 0
    result = []
    while i < len(A) and j < len(B):
        if A[i] == B[j]:
            result.append(A[i])
            i += 1
            j += 1
        elif A[i] < B[j]:
            i += 1
        else:
            j += 1
    return result # lo mismo que: [i for i in A if i in B]


def OR(A, B): # retorna los libros en los que se encuentran al menos una de las palabras
    i, j = 0, 0
    result = []
    while i < len(A) and j < len(B):
        if A[i] == B[j]:
            result.append(A[i])
            i += 1
            j += 1
        elif A[i] < B[j]:
            result.append(A[i])
            i += 1
        else:
            result.append(B[j])
            j += 1
    while i < len(A):
        result.append(A[i])
        i += 1
    while j < len(B):
        result.append(B[j])
        j += 1
    return result # lo mismo que A + list(set(B) - set(A)), pero en orden

def NOT(A): # retorna los libros en los que no se encuentra la palabra
    lista = [i for i in list_libros if i not in A]
    return lista

def ANDNOT(A,B): # retorna los libros en los que se encuentra la primera palabra pero no la segunda
    i, j = 0, 0
    result = []
    while i < len(A) and j < len(B):
        if A[i] == B[j]:
            i += 1
            j += 1
        elif A[i] < B[j]:
            result.append(A[i])
            i += 1
        else:
            j += 1
    while i < len(A):
        result.append(A[i])
        i += 1
    return result #lo mismo que [i for i in A if i not in B] pero en orden

# Ejecución
result1 = AND(L('comunidad'), L('frodo'))
result2 = OR(L('comunidad'), L('frodo'))
result3 = NOT(L('comunidad'))
result4 = ANDNOT(L('frodo'), L('gondor'))
print("Comunidad en los libros: ", L('comunidad'))
print("Frodo en los libros: ", L('frodo'))
print("Gondor en los libros: ", L('gondor'))
print("")
print(f'comunidad AND frodo: {result1}') # INTERSECCION
print(f'comunidad OR frodo: {result2}') # UNION
print(f'NOT comunidad: {result3}') # NEGACION
print(f'frodo AND-NOT gnndor: {result4}') # DIFERENCIA

print()
print('Ejemplo de consultas:\n')

#b) Probar el programa con al menos 3 consultas y al menos 3 términos
consulta1 = ANDNOT(AND(L('comunidad'), L('frodo')), L('gondor')) # [2] - [2,3,5,6] = []
print(f'(comunidad AND frodo) AND-NOT gondor: {consulta1}')

consulta2 = OR(AND(L('comunidad'), L('frodo')), L('gondor')) # [2] + [2,3,5,6] = [2,3,5,6]
print(f'(comunidad AND frodo) OR gondor: {consulta2}')

#usar las palabras gandalf, hermana y gracias
consulta3 = OR(ANDNOT(L('gandalf'), L('hermana')), L('gracias')) # ([1,2,3,5,6] - [5,6]) + [1,4,5] = [1,2,3] + [1,4,5] = [1,2,3,4,5]
print(f'(gandalf AND-NOT hermana) OR gracias: {consulta3}')

Comunidad en los libros:  [2]
Frodo en los libros:  [1, 2, 3, 4, 5, 6]
Gondor en los libros:  [2, 3, 5, 6]

comunidad AND frodo: [2]
comunidad OR frodo: [1, 2, 3, 4, 5, 6]
NOT comunidad: [1, 3, 4, 5, 6]
frodo AND-NOT gnndor: [1, 4]

Ejemplo de consultas:

(comunidad AND frodo) AND-NOT gondor: []
(comunidad AND frodo) OR gondor: [2, 3, 5, 6]
(gandalf AND-NOT hermana) OR gracias: [1, 3, 4, 5]
